__Imports__

In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from PIL import Image
import numpy as np
import cv2
import glob

c:\Users\marta\anaconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
paths=['C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset/Training',
       'C:/Users/marta/OneDrive/Desktop/Osnabruck/Implementing ANNswithTensorFlow/FinalProject/BrainMRIdataset/Testing']

In [5]:
# Function to display image
def imshow(img):
    fig,ax=plt.subplots(1,1,figsize=(8,8))
    ax.imshow(img,cmap='gray')

In [8]:
# Labels
#labels=os.listdir(paths[0])
#labels

['glioma', 'meningioma', 'notumor', 'pituitary']

In [2]:
data_dir = 'C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset'

# Define the class names
class_names = ['tumor', 'notumor']

# Define a function to get the class label for each image (one-hot encoding: 0 is no tumor; 1 tumor)
def get_label(file_path):
    parts = tf.strings.split(file_path, sep='/')
    # Check if the image is in a tumor subfolder
    is_tumor = tf.math.reduce_any(tf.equal(parts[-2], class_names))
    return tf.cond(is_tumor, lambda: 0, lambda: 1)

print(get_label('C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset/Training/glioma/Tr-gl_0010.jpg'))

#Define a function to read an image from a file path
def process_path(file_path):
    # Get the class label for this image
    label = get_label(file_path)
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    # Resizing to the default input size of DenseNet121
    img = cv2.resize(img, (224, 224))
    img = tf.convert_to_tensor(np.array(img))
    img = np.dstack([img, img, img])
    # Normalize the pixel values to be between 0 and 1
    img = img.astype('float32') / 255
    return img, label

#print(process_path(('C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset/Testing/notumor/Te-no_0012.jpg')))

# Create a list of all sub-directories of training data of all classes, which contain the images
train_ds_list = []

# A list of files from the training directory
train_ds_list.extend(glob.glob('C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset/Training/*/*.jpg'))

# Create a list of all sub-directories of testing data of all classes, which contain the images
test_ds_list = []

# A list of files from the testing directory
test_ds_list.extend(glob.glob('C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/BrainMRIdataset/Testing/*/*.jpg'))

# # Map the file paths to image and label tensors
# labeled_train_ds = []

# for files in train_ds_list:
#     labeled_train_ds.append(process_path(files))

# x, y = zip(*labeled_train_ds)
# train_ds = tf.data.Dataset.from_tensor_slices((list(x), list(y)))

# Map the file paths to image and label tensors
labeled_test_ds = []

for files in test_ds_list:
    labeled_test_ds.append(process_path(files))

x, y = zip(*labeled_test_ds)
test_ds = tf.data.Dataset.from_tensor_slices((list(x), list(y)))

# Define the batch size
batch_size = 16

# Shuffle and batch the data
#train_ds = train_ds.shuffle(buffer_size=1000).batch(batch_size)

# # Print the first batch of data
# for images, labels in train_ds.take(1):
#     print(images.shape)
#     print(labels.numpy())

1


In [3]:
print(type(test_ds))
# Print the first batch of data
for images, labels in test_ds.take(1):
    print(images.shape)
    print(labels.numpy())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>
(224, 224, 3)
1
